## Import necessary libraries

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from statsmodels.tsa.seasonal import STL, seasonal_decompose
import pymannkendall as mk
import rasterio
from rasterio.merge import merge
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## Importance function used for model generation

In [ ]:
def create_month_range(start_date, end_date):
    """
    Generate a list of monthly dates between start_date and end_date.
    Returns dates in 'YYYY-MM' format.
    """
    monthly_dates = []
    current_date = start_date
    while current_date <= end_date:
        monthly_dates.append(current_date.strftime('%Y-%m'))
        current_date += relativedelta(months=1)
    return monthly_dates

def atoi(text):
    """
    Convert text to an integer if it's a digit; otherwise, return the text.
    """
    return int(text) if text.isdigit() else text

def natural_keys(text):
    """
    Sort strings in a natural way (e.g., file1, file2, ..., file10).
    """
    return [atoi(c) for c in re.split(r'(\d+)', text)]

def get_folder_list(path):
    """
    Get a list of all subdirectories in the given path.
    """
    return [entry.name for entry in os.scandir(path) if entry.is_dir()]

def split_x_y(sequences):
    """
    Split input sequences into features (X) and targets (y).
    """
    X, y = list(), list()
    X.append(sequences[:, :-1])
    y.append(sequences[:, -1])
    return np.array(X), np.array(y)

def calculate_mean_value(tif_file):
    """
    Calculate the mean value of a raster file, excluding nodata values.
    """
    with rasterio.open(tif_file) as src:
        data = src.read(1, masked=True)
        data_without_nodata = data[~data.mask]
        mean_value = data_without_nodata.mean()
    return mean_value

def calculate_matrix(observed, predicted):
    """
    Calculate performance metrics between observed and predicted values.
    """
    RMSE = np.sqrt(np.mean((predicted - observed)**2))
    mean_observed = np.mean(observed)
    NSE = 1 - (np.sum((predicted - observed)**2) / np.sum((observed - mean_observed)**2))
    r = pearsonr(observed, predicted)
    return round(RMSE, 3), round(NSE, 3), round(r[0], 3)

def add_slope_to_filled_rasters(path_slope, imputed_path, output_folder):
    """
    Adds the slope values to the filled raster files and saves the result to the output folder.

    Parameters:
    path_slope (str): Directory path containing slope raster files.
    imputed_path (str): Directory path containing the imputed (filled) raster files.
    output_folder (str): Directory path where the output rasters will be saved.
    """
    # List all .tif files in the slope directory
    list_files = [f for f in os.listdir(path_slope) if f.endswith('.tif')]
    list_files.sort(key=natural_keys)  # Sort files naturally

    # Create output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each file
    for f in list_files: 
        datasets = []

        # Read the slope and filled raster files
        slope_file = os.path.join(path_slope, f)
        filled_file = os.path.join(imputed_path, f)

        with rasterio.open(slope_file) as sl:
            slp = sl.read(1)

        datasets.append(slp)
        with rasterio.open(filled_file) as fl:
            flp = fl.read(1)
        datasets.append(flp)

        # Create a mask for nodata values in the filled raster
        nodata_mask = (flp == fl.nodata)

        # Add the slope to the filled raster, excluding nodata values
        result = np.where(nodata_mask, flp, datasets[0] + datasets[1])

        # Update metadata with nodata information
        meta = fl.meta.copy()
        meta.update(nodata=fl.nodata)

        # Create a new GeoTIFF file to store the result
        output_file = os.path.join(output_folder, f)
        with rasterio.open(output_file, 'w', **meta) as dst:
            dst.write(result.astype(meta['dtype']), 1)

        print(f"Processed: {output_file}")

## Automated Raster-based Trend and detrended Analysis of Hydroclimatic data

In [ ]:
import os
import numpy as np
import rasterio
from rasterio import mask
from datetime import datetime
from dateutil.relativedelta import relativedelta
from stldecompose import STL
import pymannkendall as mk
import re


def data_retrieve_withna(fol_files, date_range, array):
    """
    Retrieve data with missing values for the given date range.
    Missing entries are replaced with NaN.
    """
    data_with_missing = []
    i = 0
    for file in date_range:
        if file in fol_files:
            data_with_missing.append(array[i])
            i += 1
        else:
            data_with_missing.append(np.nan)
    return data_with_missing


def stl_method(data_without_na, array_miss):
    """
    Apply STL decomposition to extract trend and seasonal components.
    Compute the linear trend using Mann-Kendall trend analysis.
    """
    stl_result = STL(data_without_na, period=12, seasonal=13, trend_deg=0, seasonal_deg=0, low_pass_deg=0, robust=True).fit()
    trend_component = stl_result.trend

    # Perform Mann-Kendall trend analysis
    mk_result = mk.original_test(trend_component)
    linear_trend = [(x + 1) * mk_result.slope + mk_result.intercept for x in range(len(array_miss))]

    # Calculate Detrended
    detrended = np.array(array_miss) - np.array(linear_trend)
    return linear_trend, detrended


# Define the path to the base directory
path = r"C:\Users\YourUsername\model"

# Get the list of subdirectories
folder_list = get_folder_list(path)

for folder in folder_list:
    # Define the path for the current folder
    fol_path = os.path.join(path, folder)
    print(f"Processing folder: {fol_path}")

    # Get the list of .tif files and sort them naturally
    fol_files = [f for f in os.listdir(fol_path) if f.endswith('.tif')]
    fol_files.sort(key=natural_keys)

    # Read reference raster metadata
    with rasterio.open(os.path.join(fol_path, fol_files[0])) as ref:
        ref_data = ref.read(1, masked=True)
        profile = ref.profile
        nodata = ref.nodata

    # Define date range for the analysis
    start_date = datetime(2002, 4, 1)
    end_date = datetime(2022, 8, 31)
    date_range = [f + ".tif" for f in create_month_range(start_date, end_date)]

    # Read all raster files into an array
    raster_array = []
    for filename in fol_files:
        with rasterio.open(os.path.join(fol_path, filename)) as src:
            input_data = src.read(1, masked=True)
            raster_array.append(input_data)

    raster_array = np.array(raster_array)
    n_sample, row, col = raster_array.shape

    # Create output directories for slope and detrended files
    slope_path = os.path.join(fol_path, "Slope")
    os.makedirs(slope_path, exist_ok=True)

    detrended_path = os.path.join(fol_path, "Detrended")
    os.makedirs(detrended_path, exist_ok=True)

    # Process each pixel in the raster data
    lt = []  # Linear trend
    dt = []  # Detrended

    for r in range(row):
        for c in range(col):
            if raster_array[0, r, c] != nodata:
                # Retrieve data for the current pixel
                array_miss = data_retrieve_withna(fol_files, date_range, raster_array[:, r, c])
                data_without_na = [x for x in array_miss if not np.isnan(x)]

                # Perform STL decomposition and compute detrended
                linear_trend, detrended = stl_method(data_without_na, array_miss)
                lt.append(linear_trend)
                dt.append(detrended)
            else:
                # Fill missing values with nodata
                array_miss = data_retrieve_withna(fol_files, date_range, raster_array[:, r, c])
                lt.append([nodata] * len(array_miss))
                dt.append(array_miss)

    # Reshape results back into raster dimensions
    lt = np.array(lt).reshape(row, col, -1)
    dt = np.array(dt).reshape(row, col, -1)

    # Save the linear trend and detrended as raster files
    for sam in range(lt.shape[-1]):
        with rasterio.open(os.path.join(slope_path, date_range[sam]), "w", **profile) as dst:
            dst.write(lt[:, :, sam], 1)
        with rasterio.open(os.path.join(detrended_path, date_range[sam]), "w", **profile) as dst:
            dst.write(dt[:, :, sam], 1)

        print(f"File {sam + 1}/{lt.shape[-1]} saved successfully.")


## Preparation of training, testing, and gap-filling datasets for a model using raster data

In [ ]:
### Configuration and initialization
# Path to the detrended model data folder
path = r'C:\Users\YourUsername\model\Detrended'
vars_list = get_folder_list(path)  # Get list of all variable folders

# Define the target variable and remove it from the variable list
target_var = 'GRACE TWSA'  # Define the exact name of the folder of target
vars_list.remove(target_var)  # Exclude GRACE TWSA from the list of input variables
input_vars = vars_list  # Remaining variables as input variables

# List and sort all target files (TIF files)
target_files = [f for f in os.listdir(os.path.join(path, target_var)) if f.endswith('.tif')]
target_files.sort(key=natural_keys)

# Get files for the first input variable
var1_files = [f for f in os.listdir(os.path.join(path, input_vars[0])) if f.endswith('.tif')]
var1_files.sort(key=natural_keys)

# Define specific files for training and testing
Data_to_find = [target_files[0], '2017-06.tif', '2018-06.tif', target_files[-1]]

# Find indices for the selected files
indices = [index for index, value in enumerate(target_files) if value in Data_to_find]

# Separate training and testing data
train_data = target_files[indices[0]:indices[1] + 1]
test_data = target_files[indices[2]:indices[3] + 1]

# Gap-filling data (first and last file for each variable)
Data_gap_find = [var1_files[0], var1_files[-1]]
indices_fill = [index for index, value in enumerate(var1_files) if value in Data_gap_find]
gap_data = var1_files[indices_fill[0]:indices_fill[-1] + 1]

# Read metadata from the first target raster
with rasterio.open(os.path.join(path, target_var, target_files[0])) as ref:
    ref_data = ref.read(1, masked=True)
    profile = ref.profile

### Data preparation
# Prepare training data as a dictionary of stacked arrays
train_array = {}
for index, filename in enumerate(train_data):
    raster_array = []
    for var in input_vars:  # Iterate over input variables
        try:
            with rasterio.open(os.path.join(path, var, filename)) as src:
                input_data = src.read(1, masked=True)  # Read raster data
                raster_array.append(input_data)
        except FileNotFoundError:
            print(f"File not found: {filename} in variable {var}")
    try:
        with rasterio.open(os.path.join(path, target_var, filename)) as src_t:
            target_data = src_t.read(1, masked=True)  # Read target data
            raster_array.append(target_data)
        train_array[index] = np.stack(raster_array, axis=0)  # Stack arrays along a new dimension
    except FileNotFoundError:
        print(f"File not found: {filename} in target variable")

# Prepare testing data (similar to training data)
test_array = {}
for index, filename in enumerate(test_data):
    raster_array = []
    for var in input_vars:
        try:
            with rasterio.open(os.path.join(path, var, filename)) as src:
                input_data = src.read(1, masked=True)
                raster_array.append(input_data)
        except FileNotFoundError:
            print(f"File not found: {filename} in variable {var}")
    try:
        with rasterio.open(os.path.join(path, target_var, filename)) as src_t:
            target_data = src_t.read(1, masked=True)
            raster_array.append(target_data)
        test_array[index] = np.stack(raster_array, axis=0)
    except FileNotFoundError:
        print(f"File not found: {filename} in target variable")

# Prepare gap-filling data (similar to training and testing)
gap_array = {}
for index, filename in enumerate(gap_data):
    raster_array = []
    for var in input_vars:
        try:
            with rasterio.open(os.path.join(path, var, filename)) as src:
                input_data = src.read(1, masked=True)
                raster_array.append(input_data)
        except FileNotFoundError:
            print(f"File not found: {filename} in variable {var}")
    try:
        gap_array[index] = np.stack(raster_array, axis=0)
    except ValueError:
        print(f"Error stacking array for gap-filling data at index {index}")


## Data standarization

In [ ]:
# Initialize arrays to store mean and standard deviation values for gap samples
gap_mean_array = []
gap_std_array = []

# Loop through each feature
for feature in range(n_features):
    # Handle all features except the last one
    if feature != n_features - 1:
        # Iterate over each row and column in the data
        for r in range(row):
            for c in range(col):
                
                gap_temp = []

                # Normalize data for training samples
                for train_sample in train_samples:
                    train_temp.append(train_array[train_sample][feature, r, c])

                # Check if the first value is not a missing value (e.g., 9999.0)
                if np.array(train_temp)[0] != 9999.0:
                    train_mean = np.array(train_temp).mean()
                    train_std = np.array(train_temp).std()

                    # Check if standard deviation is non-zero for normalization
                    if train_std != 0.0:
                        train_normalized = (train_temp - train_mean) / train_std
                        train_norm[feature, r, c] = train_normalized
                    else:  # If std is zero, only subtract the mean
                        train_normalized = (train_temp - train_mean)
                        train_norm[feature, r, c] = train_normalized
                else:
                    # If missing value, retain original values
                    train_norm[feature, r, c] = np.array(train_temp)

                # Normalize data for testing samples
                for test_sample in test_samples:
                    test_temp.append(test_array[test_sample][feature, r, c])

                if np.array(test_temp)[0] != 9999.0:
                    test_mean = np.array(test_temp).mean()
                    test_std = np.array(test_temp).std()

                    if test_std != 0.0:
                        test_normalized = (test_temp - test_mean) / test_std
                        test_norm[feature, r, c] = test_normalized
                    else:
                        test_normalized = (test_temp - test_mean)
                        test_norm[feature, r, c] = test_normalized
                else:
                    test_norm[feature, r, c] = np.array(test_temp)

                # Normalize data for gap samples
                for gap_sample in gap_samples:
                    gap_temp.append(gap_array[gap_sample][feature, r, c])

                if np.array(gap_temp)[0] != 9999.0:
                    gap_mean = np.array(gap_temp).mean()
                    gap_std = np.array(gap_temp).std()

                    if gap_std != 0.0:
                        gap_normalized = (gap_temp - gap_mean) / gap_std
                        gap_norm[feature, r, c] = gap_normalized
                    else:
                        gap_normalized = (gap_temp - gap_mean)
                        gap_norm[feature, r, c] = gap_normalized
                else:
                    gap_norm[feature, r, c] = np.array(gap_temp)

    else:  # Special handling for the last feature
        for r in range(row):
            for c in range(col):
                train_temp = []
                test_temp = []
                gap_temp = []

                # Normalize data for training samples
                for train_sample in train_samples:
                    train_temp.append(train_array[train_sample][feature, r, c])

                if np.array(train_temp)[0] != -99999.0:
                    train_mean = np.array(train_temp).mean()
                    train_std = np.array(train_temp).std()

                    if train_std != 0.0:
                        train_normalized = (train_temp - train_mean) / train_std
                        train_norm[feature, r, c] = train_normalized
                    else:
                        train_normalized = (train_temp - train_mean)
                        train_norm[feature, r, c] = train_normalized
                else:
                    train_norm[feature, r, c] = np.array(train_temp)

                # Normalize data for testing samples
                for test_sample in test_samples:
                    test_temp.append(test_array[test_sample][feature, r, c])

                if np.array(test_temp)[0] != -99999.0:
                    test_mean = np.array(test_temp).mean()
                    test_std = np.array(test_temp).std()

                    if test_std != 0.0:
                        test_normalized = (test_temp - test_mean) / test_std
                        test_norm[feature, r, c] = test_normalized
                    else:
                        test_normalized = (test_temp - test_mean)
                        test_norm[feature, r, c] = test_normalized
                else:
                    test_norm[feature, r, c] = np.array(test_temp)

# Reorganize and reshape normalized arrays for gap data
gap_norm_image = []
gap_mean_array = []
gap_std_array = []

for k in gap_norm.keys():
    gap_norm_image.append(gap_norm[k])
gap_norm_image = np.array(gap_norm_image).reshape(n_features - 1, row, col, np.array(gap_norm_image).shape[1])

for index in range(row * col):
    gap_mean_array.append((train_mean_array[index] + test_mean_array[index]) / 2)
    gap_std_array.append((train_std_array[index] + test_std_array[index]) / 2)
gap_mean_array = np.array(gap_mean_array).reshape(row, col)
gap_std_array = np.array(gap_std_array).reshape(row, col)

# Reorganize normalized training data
train_norm_image = []
for k in train_norm.keys():
    train_norm_image.append(train_norm[k])
train_norm_image = np.array(train_norm_image).reshape(n_features, row, col, np.array(train_norm_image).shape[1])

# Reorganize normalized testing data
test_norm_image = []
for k in test_norm.keys():
    test_norm_image.append(test_norm[k])
test_norm_image = np.array(test_norm_image).reshape(n_features, row, col, np.array(test_norm_image).shape[1])


## ANN model applied on Detrended data

In [ ]:
# ANN Model Start
print("*** ANN model start processing***")

# Define ANN Model Class
class FullyConnectedLayer(nn.Module):
    def __init__(self, input_size, output_size, dropout_prob):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.bn1 = nn.LayerNorm(32)  # Better for small batch sizes
        self.dropout1 = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(32, 16)
        self.dropout2 = nn.Dropout(p=dropout_prob)
        self.fc3 = nn.Linear(16, output_size)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.bn1(out)
        out = self.dropout1(out)
        out = F.relu(self.fc2(out))
        out = self.dropout2(out)
        out = self.fc3(out)
        return out


# Define no data value
nodata = -99999.0

# Initialize output storage
whole_data_fill = []

# Check if CUDA is available for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Main Training and Prediction Loop
for r in range(row):
    for c in range(col):
        if train_norm_image[-1, r, c, 0] != nodata:
            x_train_in, y_train_in = split_x_y(train_norm_image[:, r, c, :].T)
            x_test_in, y_test_in = split_x_y(test_norm_image[:, r, c, :].T)
            x_gap_in = gap_norm_image[:, r, c, :].T

            # Convert to tensors and move to device
            x_train_in = torch.tensor(x_train_in).float().to(device)
            y_train_in = torch.tensor(y_train_in.T).float().to(device)
            x_test_in = torch.tensor(x_test_in).float().to(device)
            y_test_in = torch.tensor(y_test_in.T).float().to(device)
            x_gap_in = torch.tensor(x_gap_in).float().to(device)

            # Prepare datasets and dataloaders
            train_dataset = TensorDataset(x_train_in, y_train_in)
            train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
            test_dataset = TensorDataset(x_test_in, y_test_in)
            test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

            # Initialize model
            input_size = x_train_in.shape[1]
            model = FullyConnectedLayer(input_size, output_size, dropout_prob).to(device)
            criterion = nn.MSELoss()

            # Include L2 regularization via weight_decay
            optimizer = optim.Adam(model.parameters(), lr=0.0035, weight_decay=0.01)  # L2 reg applied here

            # Training Loop
            best_val_loss = float('inf')
            patience = 10
            counter = 0

            for epoch in range(1000):
                model.train()
                for inputs, targets in train_dataloader:
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    optimizer.step()

                # Validation Loss
                model.eval()
                val_loss = 0.0
                with torch.no_grad():
                    for inputs, targets in test_dataloader:
                        outputs = model(inputs)
                        val_loss += criterion(outputs, targets).item()

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    counter = 0
                else:
                    counter += 1
                    if counter >= patience:
                        break

            # Prediction
            model.eval()
            with torch.no_grad():
                gap_preds = model(x_gap_in).cpu().numpy()
                gap_preds = gap_preds * gap_std_array[r, c] + gap_mean_array[r, c]
                whole_data_fill[r, c, :] = gap_preds

# Save the filled data
n_sample = np.array(whole_data_fill).shape[-1]
imputed_path = r"C:\Users\YourUsername\model\Results\ANN"
os.makedirs(imputed_path, exist_ok=True)

whole_data_fill = np.array(whole_data_fill).reshape(row, col, n_sample)
for sample in range(n_sample):
    temp = whole_data_fill[:, :, sample].astype(np.float32)
    with rasterio.open(imputed_path + var1_files[sample], "w", **profile) as dst:
        dst.write(temp, 1)

# Add slope to the filled rasters
path_slope = r"C:\Users\YourUsername\model\GRACE TWSA\Slope"
output_folder = os.path.join(imputed_path, "Added Slope")
add_slope_to_filled_rasters(path_slope, imputed_path, output_folder)


## SLSTM model

In [ ]:
# LSTM Model Start
print("*** SLSTM model start processing***")

# Function to convert time series to supervised learning format
def series_to_supervised(sequences, n_steps_in=1, n_steps_out=1):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]  # Input and output split
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Function to convert sequences to supervised data (gap data)
def series_to_supervised_gap(sequences, n_steps_in=1, n_steps_out=1):
    X = list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        seq_x = sequences[i:end_ix]
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(sequences):
            break
        X.append(seq_x)
    return np.array(X)


# Define no data value
nodata = -99999.0

# Initialize output storage
whole_data_fill = []

# Check if CUDA is available for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time_step = 3  # Time step for sequence length

# Loop through each grid cell
for r in range(row):
    for c in range(col):
        if train_norm_image[-1, r, c, 0].T != nodata:  # Check for valid data
            print("Processing grid cell:", r, c)

            # Prepare input and output sequences for training and testing
            x_train_in, y_train_in = series_to_supervised(train_norm_image[:, r, c, :].T, n_steps_in=time_step, n_steps_out=1)
            x_test_in, y_test_in = series_to_supervised(test_norm_image[:, r, c, :].T, n_steps_in=time_step, n_steps_out=1)
            x_gap_in = series_to_supervised_gap(gap_norm_image[:, r, c, :].T, n_steps_in=time_step)

            # Convert to PyTorch tensors
            x_train_in, y_train_in = torch.tensor(x_train_in).float(), torch.tensor(y_train_in.T).float()
            x_test_in, y_test_in = torch.tensor(x_test_in).float(), torch.tensor(y_test_in.T).float()
            x_gap_in = torch.tensor(x_gap_in).float()

            # Move tensors to device
            x_train_in, y_train_in = x_train_in.to(device), y_train_in.to(device)
            x_test_in, y_test_in = x_test_in.to(device), y_test_in.to(device)
            x_gap_in = x_gap_in.to(device)

            # Define LSTM model class
            class LSTM(nn.Module):
                def __init__(self, input_size, hidden_sizes, output_size, dropout_rates, weight_decay=0.01):
                    super(LSTM, self).__init__()
                    self.lstm_layers = nn.ModuleList([
                        nn.LSTM(input_size if i == 0 else hidden_sizes[i - 1],
                                hidden_size,
                                num_layers=2,
                                dropout=dropout_rates[i],
                                batch_first=True)
                        for i, hidden_size in enumerate(hidden_sizes)
                    ])
                    self.linear = nn.Linear(hidden_sizes[-1], output_size)

                def forward(self, x):
                    for lstm_layer in self.lstm_layers:
                        x, _ = lstm_layer(x)
                    out = self.linear(x[:, -1, :])
                    return out

            # Define model hyperparameters
            input_size = x_train_in.shape[-1]
            output_size = 1
            hidden_sizes = [150, 100]
            dropout_rates = [0.1, 0.1]
            weight_decay = 0.0001
            batch_size = 8

            # Create DataLoader
            train_dataset = TensorDataset(x_train_in, y_train_in)
            test_dataset = TensorDataset(x_test_in, y_test_in)
            train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            # Initialize model
            model = LSTM(input_size, hidden_sizes, output_size, dropout_rates, weight_decay).to(device)

            # Define loss and optimizer
            criterion = nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0035, weight_decay=weight_decay)

            # Training loop
            best_val_loss = float('inf')
            patience = 10
            counter = 0

            for epoch in range(1000):
                model.train()
                for inputs, targets in train_dataloader:
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs.squeeze(), targets.squeeze())
                    loss.backward()
                    optimizer.step()

                # Validation loop
                model.eval()
                test_loss = 0.0
                with torch.no_grad():
                    for inputs, targets in test_dataloader:
                        outputs = model(inputs)
                        val_loss = criterion(outputs.squeeze(), targets.squeeze())
                        test_loss += val_loss.item()
                avg_test_loss = test_loss / len(test_dataloader)

                # Early stopping
                if avg_test_loss < best_val_loss:
                    best_val_loss = avg_test_loss
                    counter = 0
                else:
                    counter += 1
                    if counter >= patience:
                        break

            # Predict gap data
            model.eval()
            with torch.no_grad():
                gap_preds = model(x_gap_in).cpu().numpy()
                gap_preds = gap_preds * gap_std_array[r, c] + gap_mean_array[r, c]
                whole_data_fill.append(gap_preds.ravel())
        else:
            f_nodata = [nodata for _ in range(gap_norm_image[-1, 0, 0, :].T.shape[0] - time_step + 1)]
            whole_data_fill.append(f_nodata)

# Reshape and save output
whole_data_fill = np.array(whole_data_fill).reshape(row, col, -1)

imputed_path = r"C:\Users\YourUsername\model\Results\LSTM"
os.makedirs(imputed_path, exist_ok=True)

n_sample = whole_data_fill.shape[-1]
for sample in range(n_sample):
    temp = whole_data_fill[:, :, sample].astype(np.float32)
    with rasterio.open(os.path.join(imputed_path, var1_files[sample + time_step - 1]), "w", **profile) as dst:
        dst.write(temp, 1)
        print(f"Saved: {var1_files[sample + time_step - 1]}")

# Add slope
path_slope = r"C:\Users\YourUsername\model\GRACE TWSA\Slope"
output_folder = os.path.join(imputed_path, "Added Slope")
add_slope_to_filled_rasters(path_slope, imputed_path, output_folder)


## Random Forest

In [ ]:
## Print process initiation
print("## Random Forest Processing start... ##")

# Define NoData value and initialize storage for filled data
nodata = -99999.0
whole_data_fill = []

# Loop through rows and columns of the data
for r in range(row):
    for c in range(col):
        # Check if the pixel value is not NoData
        if train_norm_image[-1, r, c, 0].T != nodata:
            # Split normalized training data into input and output
            x_train_in, y_train_in = split_x_y(train_norm_image[:, r, c, :].T)
            x_train_in, y_train_in = x_train_in.squeeze(), y_train_in.squeeze()

            # Split normalized testing data into input and output
            x_test_in, y_test_in = split_x_y(test_norm_image[:, r, c, :].T)
            x_test_in, y_test_in = x_test_in.squeeze(), y_test_in.squeeze()

            # Prepare the normalized gap data for prediction
            x_gap_in = gap_norm_image[:, r, c, :].T
            x_gap_in = x_gap_in.squeeze()

            # Train a Random Forest regressor
            rf_regressor = RandomForestRegressor(n_estimators=150, random_state=42)
            rf_regressor.fit(x_train_in, y_train_in.ravel())

            # Predict gap values
            gap_preds = rf_regressor.predict(x_gap_in)

            # Denormalize predicted values using the stored mean and standard deviation
            gap_preds = gap_preds * gap_std_array[r, c] + gap_mean_array[r, c]

            # Append the predictions for the current pixel to the storage
            whole_data_fill.append(np.array(gap_preds).ravel())
        else:
            # Handle NoData pixels by filling with NoData value
            f_nodata = [nodata for _ in range(gap_norm_image[-1, 0, 0, :].T.shape[0])]
            whole_data_fill.append(np.array(f_nodata))

# Reshape the filled data to the original dimensions
n_sample = np.array(whole_data_fill).shape[-1]
whole_data_fill = np.array(whole_data_fill).reshape(row, col, n_sample)

# Define output path for the imputed data and create directory if it doesn't exist
imputed_path = r"C:\Users\YourUsername\model\Results\RF"
os.makedirs(imputed_path, exist_ok=True)

# Save the filled data as raster files
for sample in range(n_sample):
    temp = whole_data_fill[:, :, sample].astype(np.float32)
    with rasterio.open(imputed_path + var1_files[sample], "w", **profile) as dst:
        dst.write(temp, 1)
        print(var1_files[sample])
        
# Add slope
path_slope = r"C:\Users\YourUsername\model\GRACE TWSA\Slope"
output_folder = os.path.join(imputed_path, "Added Slope")
add_slope_to_filled_rasters(path_slope, imputed_path, output_folder)
print("## Random Forest Processing completed successfully. ##")

## Support Vector Machine

In [ ]:
# Print process initiation
print("## Support Vector Processing start... ##")

# Define NoData value and initialize storage for filled data
nodata = -99999.0
whole_data_fill = []

# Loop through rows and columns of the data
for r in range(row):
    for c in range(col):
        # Check if the pixel value is not NoData
        if train_norm_image[-1, r, c, 0].T != nodata:
            # Split normalized training data into input and output
            x_train_in, y_train_in = split_x_y(train_norm_image[:, r, c, :].T)
            x_train_in, y_train_in = x_train_in.squeeze(), y_train_in.squeeze()

            # Split normalized testing data into input and output
            x_test_in, y_test_in = split_x_y(test_norm_image[:, r, c, :].T)
            x_test_in, y_test_in = x_test_in.squeeze(), y_test_in.squeeze()

            # Prepare the normalized gap data for prediction
            x_gap_in = gap_norm_image[:, r, c, :].T
            x_gap_in = x_gap_in.squeeze()

            # Train a Support Vector regressor
            svr_regressor = SVR(kernel='rbf')
            svr_regressor.fit(x_train_in, y_train_in.ravel())

            # Predict gap values
            gap_preds = svr_regressor.predict(x_gap_in)

            # Denormalize predicted values using the stored mean and standard deviation
            gap_preds = gap_preds * gap_std_array[r, c] + gap_mean_array[r, c]

            # Append the predictions for the current pixel to the storage
            whole_data_fill.append(np.array(gap_preds).ravel())
        else:
            # Handle NoData pixels by filling with NoData value
            f_nodata = [nodata for _ in range(gap_norm_image[-1, 0, 0, :].T.shape[0])]
            whole_data_fill.append(np.array(f_nodata))

# Reshape the filled data to the original dimensions
n_sample = np.array(whole_data_fill).shape[-1]
whole_data_fill = np.array(whole_data_fill).reshape(row, col, n_sample)

# Define output path for the imputed data and create directory if it doesn't exist
imputed_path = r"C:\Users\YourUsername\model\Results\SVR"
os.makedirs(imputed_path, exist_ok=True)

# Save the filled data as raster files
for sample in range(n_sample):
    temp = whole_data_fill[:, :, sample].astype(np.float32)
    with rasterio.open(imputed_path + var1_files[sample], "w", **profile) as dst:
        dst.write(temp, 1)
        print(var1_files[sample])

# Add slope
path_slope = r"C:\Users\YourUsername\model\GRACE TWSA\Slope"
output_folder = os.path.join(imputed_path, "Added Slope")
os.makedirs(output_folder, exist_ok=True)

# Assuming add_slope_to_filled_rasters is defined elsewhere
add_slope_to_filled_rasters(path_slope, imputed_path, output_folder)

print("## Support Vector Processing completed successfully. ##")


## XGBRegressor

In [ ]:
# Print process initiation
print("## XGBRegressor Processing start... ##")

# Define NoData value and initialize storage for filled data
nodata = -99999.0
whole_data_fill = []

# Loop through rows and columns of the data
for r in range(row):
    for c in range(col):
        # Check if the pixel value is not NoData
        if train_norm_image[-1, r, c, 0].T != nodata:
            # Split normalized training data into input and output
            x_train_in, y_train_in = split_x_y(train_norm_image[:, r, c, :].T)
            x_train_in, y_train_in = x_train_in.squeeze(), y_train_in.squeeze()

            # Split normalized testing data into input and output
            x_test_in, y_test_in = split_x_y(test_norm_image[:, r, c, :].T)
            x_test_in, y_test_in = x_test_in.squeeze(), y_test_in.squeeze()

            # Prepare the normalized gap data for prediction
            x_gap_in = gap_norm_image[:, r, c, :].T
            x_gap_in = x_gap_in.squeeze()
            
            # Create an XGBoost Regressor
            # Initialize the XGBRegressor with specified parameters
            xgb_regressor = XGBRegressor(
                learning_rate=0.1,  # Sets the learning rate to 0.1
                max_depth=6,        # Sets the maximum depth of a tree to 6
                n_estimators=100    # Sets the number of trees to 100
            )
            xgb_regressor.fit(x_train_in, y_train_in.ravel())

            # Predict gap values
            gap_preds = xgb_regressor.predict(x_gap_in)

            # Denormalize predicted values using the stored mean and standard deviation
            gap_preds = gap_preds * gap_std_array[r, c] + gap_mean_array[r, c]

            # Append the predictions for the current pixel to the storage
            whole_data_fill.append(np.array(gap_preds).ravel())
        else:
            # Handle NoData pixels by filling with NoData value
            f_nodata = [nodata for _ in range(gap_norm_image[-1, 0, 0, :].T.shape[0])]
            whole_data_fill.append(np.array(f_nodata))

# Reshape the filled data to the original dimensions
n_sample = np.array(whole_data_fill).shape[-1]
whole_data_fill = np.array(whole_data_fill).reshape(row, col, n_sample)

# Define output path for the imputed data and create directory if it doesn't exist
imputed_path = r"C:\Users\YourUsername\model\Results\XGB"
os.makedirs(imputed_path, exist_ok=True)

# Save the filled data as raster files
for sample in range(n_sample):
    temp = whole_data_fill[:, :, sample].astype(np.float32)
    with rasterio.open(os.path.join(imputed_path, var1_files[sample]), "w", **profile) as dst:
        dst.write(temp, 1)
        print(var1_files[sample])

# Add slope
path_slope = r"C:\Users\YourUsername\model\GRACE TWSA\Slope"
output_folder = os.path.join(imputed_path, "Added Slope")
os.makedirs(output_folder, exist_ok=True)

# Assuming add_slope_to_filled_rasters is defined elsewhere
add_slope_to_filled_rasters(path_slope, imputed_path, output_folder)

print("## XGBRegressor Processing completed successfully. ##")


## Calculate performance metrices 

In [ ]:
# Define the model name
model_name = "ANN"

# Define paths
path_GRACE = r"C:\Users\YourUsername\model\GRACE TWSA"
path_model = r"C:\Users\YourUsername\model\Results\\" + model_name + "\\Added Slope"
save_path = r"C:\Users\YourUsername\model\Testing result\\" + model_name + "\\"

# Create save directory if not exists
os.makedirs(save_path, exist_ok=True)

# Get sorted list of files in GRACE folder
files_list = [f for f in os.listdir(path_GRACE) if f.endswith('.tif')]
files_list.sort(key=natural_keys)

# Identify the relevant subset of files
Data_to_find = ['2018-10.tif', files_list[-1]]
indices = [index for index, value in enumerate(files_list) if value in Data_to_find]
matrics_data = files_list[indices[0]:indices[1] + 1]

# Read reference raster for metadata
with rasterio.open(os.path.join(path_GRACE, files_list[0])) as ref:
    ref_data = ref.read(1, masked=True)
    profile = ref.profile

# Initialize arrays for GRACE and model data
raster_array_GRACE = []
raster_array_model = []

# Read data for the defined files
for filename in matrics_data:
    with rasterio.open(os.path.join(path_GRACE, filename)) as src_o:
        GRACE_data = src_o.read(1, masked=True)
        raster_array_GRACE.append(GRACE_data)

    with rasterio.open(os.path.join(path_model, filename)) as src_p:
        predicted_data = src_p.read(1, masked=True)
        raster_array_model.append(predicted_data)

# Convert data arrays to numpy
n_sample, row, col = np.array(raster_array_GRACE).shape

# Initialize metrics arrays
RMSE_array = []
NSE_array = []
PCC_array = []

# Compute metrics for each pixel
for r in range(row):
    for c in range(col):
        if np.array(raster_array_GRACE)[0, r, c] != -99999.0:
            observed_temp = []
            predicted_temp = []

            # Collect observed and predicted values for all samples
            for sample in range(n_sample):
                observed_temp.append(np.array(raster_array_GRACE)[sample, r, c])
                predicted_temp.append(np.array(raster_array_model)[sample, r, c])

            # Calculate metrics for the current pixel
            rmse, nse, PCC = calculate_matrix(np.array(observed_temp), np.array(predicted_temp))
            RMSE_array.append(rmse)
            NSE_array.append(nse)
            PCC_array.append(PCC)
        else:
            # Assign NoData values for invalid pixels
            RMSE_array.append(np.array(raster_array_GRACE)[0, r, c])
            NSE_array.append(np.array(raster_array_GRACE)[0, r, c])
            PCC_array.append(np.array(raster_array_GRACE)[0, r, c])

# Reshape metric arrays to raster dimensions
test_rmse = np.array(RMSE_array).reshape(row, col)
test_nse = np.array(NSE_array).reshape(row, col)
test_r = np.array(PCC_array).reshape(row, col)

# Save metrics as raster files
with rasterio.open(save_path + model_name + '_test_rmse.tif', "w", **profile) as dst:
    dst.write(test_rmse, 1)

with rasterio.open(save_path + model_name + '_test_nse.tif', "w", **profile) as dst:
    dst.write(test_nse, 1)

with rasterio.open(save_path + model_name + '_test_r.tif', "w", **profile) as dst:
    dst.write(test_r, 1)

print("File save Successfully")
